In [9]:
import pandas as pd
import utils
from bs4 import BeautifulSoup

In [49]:
#This section is for downloading Energy Price Data

# FOOTHILL_1_N001 - corresponds to San Luis Ob Solar data

URL = "http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_LMP&startdatetime=20161101T07:00-0000&enddatetime=20171101T07:00-0000&version=1&market_run_id=DAM&node=FOOTHILL_1_N001"

#Need to fix this to get unzipped filename even when already exists
utils.remove_data(URL,'zip')
fn = utils.get_data(URL,'zip')

with open(fn, "r") as src:
    soup = BeautifulSoup(src, "xml")

print(soup.prettify(:5000))

Removed 36c2ef9.zip
Extracting zip
<?xml version="1.0" encoding="utf-8"?>
<OASISReport xmlns="http://www.caiso.com/soa/OASISReport_v1.xsd">
 <MessageHeader>
  <TimeDate>
   2017-11-09T23:30:58-00:00
  </TimeDate>
  <Source>
   OASIS
  </Source>
  <Version>
   v20131201
  </Version>
 </MessageHeader>
 <MessagePayload>
  <RTO>
   <name>
    CAISO
   </name>
   <REPORT_ITEM>
    <REPORT_HEADER>
     <SYSTEM>
      OASIS
     </SYSTEM>
     <TZ>
      PPT
     </TZ>
     <REPORT>
      PRC_LMP
     </REPORT>
     <MKT_TYPE>
      DAM
     </MKT_TYPE>
     <UOM>
      US$/MWh
     </UOM>
     <INTERVAL>
      ENDING
     </INTERVAL>
     <SEC_PER_INTERVAL>
      3600
     </SEC_PER_INTERVAL>
    </REPORT_HEADER>
    <REPORT_DATA>
     <DATA_ITEM>
      LMP_PRC
     </DATA_ITEM>
     <RESOURCE_NAME>
      FOOTHILL_1_N001
     </RESOURCE_NAME>
     <OPR_DATE>
      2013-09-19
     </OPR_DATE>
     <INTERVAL_NUM>
      1
     </INTERVAL_NUM>
     <INTERVAL_START_GMT>
      2013-09-19T07:00:00-

In [79]:
soup.OASISReport
items = list(soup.OASISReport.findChildren("INTERVAL_NUM"))
items

[<INTERVAL_NUM>1</INTERVAL_NUM>,
 <INTERVAL_NUM>11</INTERVAL_NUM>,
 <INTERVAL_NUM>10</INTERVAL_NUM>,
 <INTERVAL_NUM>13</INTERVAL_NUM>,
 <INTERVAL_NUM>14</INTERVAL_NUM>,
 <INTERVAL_NUM>17</INTERVAL_NUM>,
 <INTERVAL_NUM>3</INTERVAL_NUM>,
 <INTERVAL_NUM>16</INTERVAL_NUM>,
 <INTERVAL_NUM>21</INTERVAL_NUM>,
 <INTERVAL_NUM>4</INTERVAL_NUM>,
 <INTERVAL_NUM>22</INTERVAL_NUM>,
 <INTERVAL_NUM>2</INTERVAL_NUM>,
 <INTERVAL_NUM>7</INTERVAL_NUM>,
 <INTERVAL_NUM>15</INTERVAL_NUM>,
 <INTERVAL_NUM>18</INTERVAL_NUM>,
 <INTERVAL_NUM>23</INTERVAL_NUM>,
 <INTERVAL_NUM>5</INTERVAL_NUM>,
 <INTERVAL_NUM>9</INTERVAL_NUM>,
 <INTERVAL_NUM>12</INTERVAL_NUM>,
 <INTERVAL_NUM>19</INTERVAL_NUM>,
 <INTERVAL_NUM>6</INTERVAL_NUM>,
 <INTERVAL_NUM>8</INTERVAL_NUM>,
 <INTERVAL_NUM>20</INTERVAL_NUM>,
 <INTERVAL_NUM>24</INTERVAL_NUM>,
 <INTERVAL_NUM>2</INTERVAL_NUM>,
 <INTERVAL_NUM>22</INTERVAL_NUM>,
 <INTERVAL_NUM>18</INTERVAL_NUM>,
 <INTERVAL_NUM>4</INTERVAL_NUM>,
 <INTERVAL_NUM>7</INTERVAL_NUM>,
 <INTERVAL_NUM>11</INTERVA

In [85]:
df = pd.DataFrame([
    {
        
        "LMP" : i.VALUE.text if i.DATA_ITEM.text == 'LMP_PRC' else None,
        #"Energy" : i.VALUE.text if i.DATA_ITEM.text == 'LMP_ENE_PRC',
        #"Loss" : i.VALUE.text, if i.DATA_ITEM.text == 'LMP_LOSS_PRC',
        #"Congestion" : i.VALUE.text, if i.DATA_ITEM.text == 'LMP_CONG_PRC',
        "time" : i.INTERVAL_START_GMT.text}
for i in soup.OASISReport.findChildren("REPORT_DATA")]
)

df.dropna(axis=0, how='any')

,LMP,time
0,34.71202,2013-09-19T07:00:00-00:00
1,40.05189,2013-09-19T17:00:00-00:00
2,38.88823,2013-09-19T16:00:00-00:00
3,40.96826,2013-09-19T19:00:00-00:00
4,44.03239,2013-09-19T20:00:00-00:00
5,52.94277,2013-09-19T23:00:00-00:00
6,32.59409,2013-09-19T09:00:00-00:00
7,48.78539,2013-09-19T22:00:00-00:00
8,49.1757,2013-09-20T03:00:00-00:00
9,32.17771,2013-09-19T10:00:00-00:00


In [54]:
df = pd.DataFrame([
    {
        "time" : i.INTERVAL_START_GMT.text
        if i.DATA_ITEM.text == 'LMP_PRC'
            "LMP" : i.VALUE.text
        if i.DATA_ITEM.text == 'LMP_ENE_PRC'
            "Energy" : i.VALUE.text
        if i.DATA_ITEM.text == 'LMP_LOSS_PRC'
            "Loss" : i.VALUE.text
        if i.DATA_ITEM.text == 'LMP_CONG_PRC'
            "Congestion" : i.VALUE.text
        
    
    }
for i in soup.OASISReport.findChildren("REPORT_DATA")]
)
df

SyntaxError: invalid syntax (<ipython-input-54-73caad9f0082>, line 5)

In [ ]:
# Solar Data 
http://rredc.nrel.gov/solar/old_data/nsrdb/1991-2010/data/hourly/722897/722897_2010_solar.csv

In [3]:
utils.remove_data(URL,'zip')

Removed 36c2ef9.zip


In [3]:
fn

'20130919_20130920_PRC_LMP_DAM_20171109_12_46_35_v1.xml'